# THIS IS JUST A DRAFT NOTEBOOK

# Step 0 : Prepare my environment

In [1]:
import os

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF
import nltk
import pathlib # to get the file extension
import docx # deals with the word files
import PyPDF2 # deals with pdf files
import openpyxl # deals with xlsx files
from openpyxl import load_workbook
import csv
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.corpus.reader import textwrap


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Step 1 : Parser - Load my text data
Deals with a directory of mixed file types (PDF, docx, xlsx, txt), extracts text from each document and add it to a list

In [2]:
# Set the path to the directory containing the documents
doc_dir = "data"

# Create an empty list to store the document texts
doc_texts = []

# Loop over the files in the document directory
for filename in os.listdir(doc_dir):
    # Check if the file is a text file
    ex = pathlib.Path(filename).suffix
    
    if ex == ".pdf":
        # Read the contents of the PDF file
        with open(os.path.join(doc_dir, filename), 'rb') as f:
            pdf = PyPDF2.PdfReader(f)
            # Extract the text from the PDF document
            text = '\n'.join([pdf.pages[i].extract_text() for i in range(len(pdf.pages))])
            doc_texts.append(text)

    elif ex == ".docx":
      # Load the Word document
      doc = docx.Document(os.path.join(doc_dir, filename))
      # Extract the text from the document
      text = '\n'.join([para.text for para in doc.paragraphs])
      doc_texts.append(text)

    elif ex in ['.xls', '.xlsx']:
      # Load the entire workbook
      wb = load_workbook(os.path.join(doc_dir, filename))
      # Loop over each sheet in the workbook
      for worksheet in wb.worksheets:
          # Extract the text from each cell in the sheet
          xlsx_text = ""
          for row in worksheet.iter_rows(values_only=True):
              for cell in row:
                  if cell is not None:
                      xlsx_text += str(cell) + " "
      doc_texts.append(xlsx_text)


    elif ex == ".csv":
      # Open the CSV file and read the contents
      with open(os.path.join(doc_dir, filename), 'r') as f:
        reader = csv.reader(f)
        # Extract the text from each row in the CSV file
        csv_text = ""
        for row in reader:
            csv_text += ",".join(row) + " "
        doc_texts.append(csv_text)

    if ex == ".txt": #(text file)
      # Read the contents of the file
      with open(os.path.join(doc_dir, filename), 'r', encoding='utf-8') as f:
          text = f.read()
          doc_texts.append(text)

        
# Create a dictionary with one key 'text' and a list of document texts as values
data = {'text': doc_texts} # I can add the title and other features later
# Convert the dictionary to a Pandas DataFrame
df = pd.DataFrame(data)
df

,text
0,Accor IMEAT - Livre des proc édures et des po...
1,Accor Cyber -Attack and Data Breach Response ...
2,Information Technology \nGENERAL IT SECURIT...
3,73.35%\n60\n1\nAccor IT Standards Audit MEA v0...
4,"No response from Hotel IT \nafter 12 hours, co..."
5,\n\n\n\n\n\n\n\n\n\n\n1. Objectif\nLe plan de ...
6,\n\n\n\n\n\n\n\n\n\n\n1. Objectif\nLe plan d'u...
7,\n\n\n\n\n\n\n\n\n\n\n1. Objectif \nLe systèm...
8,\n\n\n\n\n\n\n\n\n\n\n1. Objectif\nLe plan d’u...
9,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n ...


# Step 2 : Data cleaning and preprocessing

## 2.1 Stopwords Removal

In [4]:
# check the list of stopwords
StopWords = stopwords.words('english')
#StopWords = StopWords + "any other list of words I want to add to my stop words list"
print(StopWords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [3]:
import torch
from transformers import pipeline

c:\Users\Asus\Documents\KPMG\PFE\clarITy\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
summarizer = pipeline("summarization")
# split second row of dataframe into smaller chunks
text = df.iloc[1]['text']
chunk_size = 10
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# generate summary for each chunk
summaries = []
for chunk in chunks:
    summary = summarizer(chunk, max_length=17, min_length=5, do_sample=False)
    summaries.append(summary[0]['summary_text'])

# concatenate summaries into a single string
summary_text = ' '.join(summaries)

# print summary text
print(summary_text)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


ValueError: Could not load model sshleifer/distilbart-cnn-12-6 with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSeq2SeqLM'>, <class 'transformers.models.bart.modeling_bart.BartForConditionalGeneration'>).

In [5]:
# removing stopwords
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in StopWords]))
# The lambda function tokenizes the text by splitting on whitespace, 
# and then removes any word that appears in the stop_words set.
df

,text
0,Accor IMEAT - Livre des proc édures et des pol...
1,Accor Cyber -Attack Data Breach Response Plan ...
2,Information Technology GENERAL SECURITY POLICY...
3,73.35% 60 1 Accor Standards Audit MEA v0.2 Moh...
4,"response Hotel 12 hours, contact:IT Support (O..."
5,1. Objectif Le plan de reprise après un désast...
6,1. Objectif Le plan d'urgence des cartes-clés ...
7,1. Objectif Le système de gestion (PMS) est le...
8,1. Objectif Le plan d’urgence pour les points ...
9,MEA Technical Competency Framework Version 1 M...


## 2.2 Punctuation Removal

In [5]:
# replace any character that is not a word character (\w) or whitespace (\s) with an empty string
df['text'] = df['text'].str.replace('[^\w\s]|[\d]','')
df

C:\Users\Asus\AppData\Local\Temp\ipykernel_11292\2828466533.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^\w\s]|[\d]','')


,text
0,Accor IMEAT Livre des proc édures et des poli...
1,Accor Cyber Attack Data Breach Response Plan V...
2,Information Technology GENERAL SECURITY POLICY...
3,Accor Standards Audit MEA v Mohamed Slama ...
4,response Hotel hours contactIT Support Office...
5,Objectif Le plan de reprise après un désastre...
6,Objectif Le plan durgence des cartesclés déta...
7,Objectif Le système de gestion PMS est le sys...
8,Objectif Le plan durgence pour les points de ...
9,MEA Technical Competency Framework Version ME...


# Step 3 : Vectorization : 
  TF-IDF Vectorizer

In [6]:
df['text'].isnull().sum()

0

In [7]:
# initialize my tf-idf vectorizer
vectorizer = TfidfVectorizer(lowercase=True, max_df=0.8, min_df=7, max_features=100, ngram_range=(1,3), stop_words="english")

# apply my vectorizer to my dataframe
vectors_matrix= vectorizer.fit_transform(df['text'])

# convert "vectors_matrix" to dataframe
tfidf_df = pd.DataFrame(vectors_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Concatenate my original df with tfidf_df
final_df = pd.concat([df, tfidf_df], axis=1)

final_df

,text,accor,annually,available,based,business,components,computer,configuration,contact,...,status,support,systems,team,test,tests,use,used,using,version
0,Accor IMEAT Livre des proc édures et des poli...,0.130255,0.000000,0.000000,0.000000,0.000973,0.000000,0.000000,0.000648,0.002269,...,0.000000,0.004865,0.000000,0.000000,0.000912,0.001734,0.000304,0.000000,0.000000,0.002918
1,Accor Cyber Attack Data Breach Response Plan V...,0.167151,0.004786,0.014358,0.033502,0.062647,0.004786,0.043075,0.000000,0.067122,...,0.004786,0.020982,0.100507,0.264012,0.025178,0.000000,0.151068,0.057433,0.033502,0.134244
2,Information Technology GENERAL SECURITY POLICY...,0.199025,0.024423,0.012212,0.024423,0.051378,0.018317,0.024423,0.028543,0.005709,...,0.012212,0.005353,0.109904,0.045669,0.021414,0.006106,0.037474,0.012212,0.006106,0.114173
3,Accor Standards Audit MEA v Mohamed Slama ...,0.125412,0.032319,0.237003,0.032319,0.191372,0.215457,0.064637,0.010072,0.020144,...,0.032319,0.094454,0.129274,0.070505,0.085009,0.021546,0.198354,0.096956,0.107729,0.080577
4,response Hotel hours contactIT Support Office...,0.065178,0.000000,0.000000,0.000000,0.078520,0.000000,0.000000,0.000000,0.863716,...,0.000000,0.073634,0.000000,0.157039,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Objectif Le plan de reprise après un désastre...,0.059981,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.036129,0.000000,...,0.000000,0.050821,0.000000,0.000000,0.101642,0.019321,0.000000,0.000000,0.000000,0.000000
6,Objectif Le plan durgence des cartesclés déta...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Objectif Le système de gestion PMS est le sys...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.143017,0.000000,...,0.076483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Objectif Le plan durgence pour les points de ...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017340,...,0.000000,0.000000,0.000000,0.000000,0.065043,0.018546,0.000000,0.000000,0.000000,0.000000
9,MEA Technical Competency Framework Version ME...,0.205460,0.013933,0.013933,0.041800,0.208434,0.027867,0.013933,0.013027,0.000000,...,0.000000,0.342061,0.264733,0.364759,0.012216,0.041800,0.012216,0.013933,0.027867,0.013027


# Language Detection

langdetect : python library

In [13]:
from langdetect import detect
detect("this is english")

'en'

# Random Ideas

In [10]:
detect("this is english")

'en'

Translator

In [8]:
df.iloc[9, 0]

'MEA Technical Competency Framework Version  MEAIT Technical Competency Framework  Table contents  Purpose    Scope    technical competencies    technical competency framework    Technical Competency Framework Structure    Applications technical competencies    Learning Development    Performance Management    Succession Planning    Recruitment Selection   Appendix A Technical competency framework table   MEAIT Technical Competency Framework   Purpose document functions guideline determine kn owledge skills andor abilities Accor hotelbased team members India Middle East Africa Based team members technical competencies Accor support th em grow careers expand skill sets several ini tiatives learning development succession planning performance management well recrui tment selection support team members acquiring devel oping necessary knowledge skills  abilities required perform thei r specific jobs aims helping line managers employees familiarize themsel ves concept technical competencies

In [1]:
from google.cloud import translate_v2 as translate

# Initialize the translation client
translate_client = translate.Client()

# Define the text to be translated and the target language
text = 'Hello, how are you?'
target = 'fr'

# Use the translation client to translate the text
translation = translate_client.translate(text, target_language=target)

# Print the translation
print(translation['input'])
print(translation['translatedText'])

ModuleNotFoundError: No module named 'google'

In [ ]:
from summarizer.sbert import SBertSummarizer

In [ ]:
body = 'Text body that you want to summarize with BERT'
model = SBertSummarizer('paraphrase-MiniLM-L6-v2')
result = model(body, num_sentences=3)
print (result)

In [ ]:
import centroid_summarizer
from nltk.tokenize import word_tokenize, sent_tokenize

# Uncomment this if you want to see a ton of debugging messages:
# centroid_summarizer.logger.setLevel(10)


# Text from the proof-of-concept code at
# https://gist.github.com/vinimonteiro/3898ce27023ec4241c4879dac67ca27d
doc = 'In an attempt to build an AI-ready workforce, Microsoft announced ' \
'Intelligent Cloud Hub which has been launched to empower the next generation ' \
'of students with AI-ready skills. Envisioned as a three-year collaborative ' \
'program, Intelligent Cloud Hub will support around 100 institutions with AI ' \
'infrastructure, course content and curriculum, developer support, development ' \
'tools and give students access to cloud and AI services. As part of the ' \
'program, the Redmond giant which wants to expand its reach and is planning ' \
'to build a strong developer ecosystem in India with the program will set up ' \
'the core AI infrastructure and IoT Hub for the selected campuses. The company ' \
'will provide AI development tools and Azure AI services such as Microsoft ' \
'Cognitive Services, Bot Services and Azure Machine Learning. According to ' \
'Manish Prakash, Country General Manager-PS, Health and Education, Microsoft ' \
'India, said, "With AI being the defining technology of our time, it is ' \
'transforming lives and industry and the jobs of tomorrow will require a ' \
'different skillset. This will require more collaborations and training and ' \
'working with AI. That’s why it has become more critical than ever for ' \
'educational institutions to integrate new cloud and AI technologies. The ' \
'program is an attempt to ramp up the institutional set-up and build capabilities ' \
'among the educators to educate the workforce of tomorrow." The program aims to ' \
'build up the cognitive skills and in-depth understanding of developing ' \
'intelligent cloud connected solutions for applications across industry. Earlier ' \
'in April this year, the company announced Microsoft Professional Program In AI ' \
'as a learning track open to the public. The program was developed to provide ' \
'job ready skills to programmers who wanted to hone their skills in AI and data ' \
'science with a series of online courses which featured hands-on labs and expert ' \
'instructors as well. This program also included developer-focused AI school ' \
'that provided a bunch of assets to help build AI skills.'

raw_sentences = sent_tokenize(doc)

clean = list(centroid_summarizer.simple_clean(raw_sentences))

cbs = centroid_summarizer.CentroidBOWSummarizer()

print("\nBag-of-words summary:\n")
print(" ".join(
    list(cbs.summarize(
        raw_sentences,
        [ " ".join(_) for _ in clean ]
    ))
),end="\n\n")


from gensim.models import Word2Vec
model = Word2Vec(clean, min_count=1)

cws = centroid_summarizer.CentroidWordEmbeddingsSummarizer(
    model
)

print("Embedding summary:\n")
print(" ".join(
    list(cws.summarize(
        raw_sentences,
        [ " ".join(_) for _ in clean ]
    ))
),end="\n\n")